In [1]:
%cd /content/drive/MyDrive/Colab/pydomains/models/latest/

/content/drive/MyDrive/Colab/pydomains/models/latest


In [81]:
import pandas as pd
import numpy as np

from sklearn.isotonic import IsotonicRegression

In [5]:
df = pd.read_csv('final.csv')

In [6]:
df.head()

,Unnamed: 0,site,actual,predicted,actual_labels,pred_labels,predicted_prob,adv,aggressive,alcohol,...,updatesites,urlshortener,violence,warez,weapons,webmail,webphone,webradio,webtv,scaled_probs
0,0,neptunesoccer,recreation,recreation,33,33,0.040239,0.018059,0.018040,0.018040,...,0.018040,0.018044,0.018040,0.018040,0.018048,0.018055,0.018040,0.018040,0.018040,0.755985
1,1,carena-girls,porn,recreation,31,33,0.027002,0.018218,0.018144,0.018174,...,0.018140,0.018175,0.018139,0.018156,0.018143,0.018237,0.018140,0.018173,0.018145,0.425040
2,2,natlhistoricseaport,recreation,gamble,33,14,0.022758,0.018316,0.018198,0.018306,...,0.018218,0.018243,0.018180,0.018228,0.018254,0.018275,0.018255,0.018183,0.018190,0.318959
3,3,gta-sanandreas,hobby,hobby,17,17,0.048474,0.017951,0.017951,0.017951,...,0.017951,0.017951,0.017951,0.017951,0.017951,0.017951,0.017951,0.017951,0.017951,0.961859
4,4,radionaba,webradio,webradio,52,52,0.029569,0.018177,0.018120,0.018119,...,0.018122,0.018192,0.018119,0.018131,0.018119,0.018155,0.018119,0.029569,0.018148,0.489214


In [7]:
df = df.drop(['Unnamed: 0'], axis=1)

In [8]:
temp_df = df.copy()

In [9]:
temp_df['y'] = temp_df['actual'] == 'porn'

In [10]:
temp_df['y'] = temp_df['y'].astype(int)

In [11]:
temp_df.head()

,site,actual,predicted,actual_labels,pred_labels,predicted_prob,adv,aggressive,alcohol,anonvpn,...,urlshortener,violence,warez,weapons,webmail,webphone,webradio,webtv,scaled_probs,y
0,neptunesoccer,recreation,recreation,33,33,0.040239,0.018059,0.018040,0.018040,0.018040,...,0.018044,0.018040,0.018040,0.018048,0.018055,0.018040,0.018040,0.018040,0.755985,0
1,carena-girls,porn,recreation,31,33,0.027002,0.018218,0.018144,0.018174,0.018134,...,0.018175,0.018139,0.018156,0.018143,0.018237,0.018140,0.018173,0.018145,0.425040,1
2,natlhistoricseaport,recreation,gamble,33,14,0.022758,0.018316,0.018198,0.018306,0.018401,...,0.018243,0.018180,0.018228,0.018254,0.018275,0.018255,0.018183,0.018190,0.318959,0
3,gta-sanandreas,hobby,hobby,17,17,0.048474,0.017951,0.017951,0.017951,0.017951,...,0.017951,0.017951,0.017951,0.017951,0.017951,0.017951,0.017951,0.017951,0.961859,0
4,radionaba,webradio,webradio,52,52,0.029569,0.018177,0.018120,0.018119,0.018119,...,0.018192,0.018119,0.018131,0.018119,0.018155,0.018119,0.029569,0.018148,0.489214,0


In [14]:
def calibrate_data(label_name, p_df):
  temp_df = p_df.copy()
  temp_df['y'] = temp_df['actual'] == label_name
  temp_df['y'] = temp_df['y'].astype(int)
  X = temp_df[label_name].to_numpy()
  y = temp_df['y'].to_numpy()
  return X, y

In [16]:
X, y = calibrate_data('porn', df)

In [17]:
from sklearn.isotonic import IsotonicRegression

In [62]:
iso_reg = IsotonicRegression().fit(X, y)

In [63]:
X[:5]

array([0.01827471, 0.02431207, 0.01881713, 0.01795125, 0.01826977])

In [64]:
iso_reg.predict(X[:5])

array([0.02012712, 0.40410639, 0.04857013, 0.00185225, 0.02012712])

In [65]:
iso_reg.transform(X[:5])

array([0.02012712, 0.40410639, 0.04857013, 0.00185225, 0.02012712])

In [29]:
df['porn_i'] = iso_reg.transform(X)

In [30]:
p_df = df[['site','actual','predicted','predicted_prob','porn','porn_i']]

In [31]:
p_df.head()

,site,actual,predicted,predicted_prob,porn,porn_i
0,neptunesoccer,recreation,recreation,0.040239,0.018275,0.020127
1,carena-girls,porn,recreation,0.027002,0.024312,0.404106
2,natlhistoricseaport,recreation,gamble,0.022758,0.018817,0.048570
3,gta-sanandreas,hobby,hobby,0.048474,0.017951,0.001852
4,radionaba,webradio,webradio,0.029569,0.018270,0.020127


In [32]:
p_df[p_df['actual'] == 'porn']

,site,actual,predicted,predicted_prob,porn,porn_i
1,carena-girls,porn,recreation,0.027002,0.024312,0.404106
19,crazysex,porn,porn,0.039957,0.039957,0.897716
26,saharagetsdirty,porn,recreation,0.026341,0.019954,0.130787
32,247pantyhose,porn,porn,0.048601,0.048601,0.992768
37,erotikstart,porn,porn,0.048240,0.048240,0.991029
...,...,...,...,...,...,...
263584,usasextoy,porn,porn,0.048551,0.048551,0.992768
263586,viciosascaseras,porn,porn,0.047873,0.047873,0.983831
263588,garotavirtual.com,porn,porn,0.045933,0.045933,0.962932
263597,jades,porn,news,0.030875,0.018335,0.021311


In [33]:
%mkdir isotonic_regression

In [34]:
%cd isotonic_regression/

/content/drive/My Drive/Colab/pydomains/models/latest/isotonic_regression


In [56]:
import joblib
joblib.dump(iso_reg, "porn.sav")

['porn.sav']

In [57]:
c_m = joblib.load("porn.sav")

In [60]:
c_m.transform(X)

array([0.02012712, 0.40410639, 0.04857013, ..., 0.        , 0.00602792,
       0.01921845])

In [61]:
iso_reg.transform(X)

array([0.02012712, 0.40410639, 0.04857013, ..., 0.        , 0.00602792,
       0.01921845])

In [84]:
def train_save(X, y, label_name):
  iso_reg = IsotonicRegression().fit(X, y)
  joblib.dump(iso_reg, f"{label_name}.sav")

In [85]:
labels = ['adv', 'aggressive', 'alcohol', 'anonvpn',
       'automobile', 'costtraps', 'dating', 'downloads', 'drugs', 'dynamic',
       'education', 'finance', 'fortunetelling', 'forum', 'gamble',
       'government', 'hacking', 'hobby', 'homestyle', 'hospitals',
       'imagehosting', 'isp', 'jobsearch', 'library', 'military', 'models',
       'movies', 'music', 'news', 'podcasts', 'politics', 'porn', 'radiotv',
       'recreation', 'redirector', 'religion', 'remotecontrol', 'ringtones',
       'science', 'searchengines', 'sex', 'shopping', 'socialnet', 'spyware',
       'tracker', 'updatesites', 'urlshortener', 'violence', 'warez',
       'weapons', 'webmail', 'webphone', 'webradio', 'webtv']

In [86]:
for label in labels:
  X, y = calibrate_data(label, df)
  train_save(X, y, label)

In [87]:
label_model = {}

for label in labels:
  model = joblib.load(f"{label}.sav")
  label_model[label] = model

In [88]:
for label in labels:
  X = df[label].to_numpy()
  df[f'{label}_i'] = label_model[label].transform(X)

In [89]:
df.head()

,site,actual,predicted,actual_labels,pred_labels,predicted_prob,adv,aggressive,alcohol,anonvpn,...,tracker_i,updatesites_i,urlshortener_i,violence_i,warez_i,weapons_i,webmail_i,webphone_i,webradio_i,webtv_i
0,neptunesoccer,recreation,recreation,33,33,0.040239,0.018059,0.018040,0.018040,0.018040,...,0.000015,0.000000,0.000145,0.000094,0.000229,0.000376,0.000209,0.000024,0.000219,0.000098
1,carena-girls,porn,recreation,31,33,0.027002,0.018218,0.018144,0.018174,0.018134,...,0.000658,0.000132,0.001177,0.000314,0.000406,0.000429,0.001650,0.000024,0.001707,0.000490
2,natlhistoricseaport,recreation,gamble,33,14,0.022758,0.018316,0.018198,0.018306,0.018401,...,0.005998,0.005170,0.001892,0.000752,0.004912,0.002587,0.001879,0.009009,0.001707,0.001827
3,gta-sanandreas,hobby,hobby,17,17,0.048474,0.017951,0.017951,0.017951,0.017951,...,0.000010,0.000000,0.000000,0.000000,0.000000,0.000085,0.000069,0.000000,0.000050,0.000011
4,radionaba,webradio,webradio,52,52,0.029569,0.018177,0.018120,0.018119,0.018119,...,0.000015,0.000063,0.001177,0.000170,0.000365,0.000376,0.001009,0.000024,0.578431,0.000490


In [90]:
common_cols = ['site', 'actual', 'predicted', 'predicted_prob']

In [91]:
uncalibrated = df[common_cols + labels]

In [92]:
uncalibrated.head()

,site,actual,predicted,predicted_prob,adv,aggressive,alcohol,anonvpn,automobile,costtraps,...,tracker,updatesites,urlshortener,violence,warez,weapons,webmail,webphone,webradio,webtv
0,neptunesoccer,recreation,recreation,0.040239,0.018059,0.018040,0.018040,0.018040,0.018056,0.018040,...,0.018042,0.018040,0.018044,0.018040,0.018040,0.018048,0.018055,0.018040,0.018040,0.018040
1,carena-girls,porn,recreation,0.027002,0.018218,0.018144,0.018174,0.018134,0.018154,0.018148,...,0.018156,0.018140,0.018175,0.018139,0.018156,0.018143,0.018237,0.018140,0.018173,0.018145
2,natlhistoricseaport,recreation,gamble,0.022758,0.018316,0.018198,0.018306,0.018401,0.018421,0.018204,...,0.018324,0.018218,0.018243,0.018180,0.018228,0.018254,0.018275,0.018255,0.018183,0.018190
3,gta-sanandreas,hobby,hobby,0.048474,0.017951,0.017951,0.017951,0.017951,0.017951,0.017951,...,0.017951,0.017951,0.017951,0.017951,0.017951,0.017951,0.017951,0.017951,0.017951,0.017951
4,radionaba,webradio,webradio,0.029569,0.018177,0.018120,0.018119,0.018119,0.018120,0.018120,...,0.018126,0.018122,0.018192,0.018119,0.018131,0.018119,0.018155,0.018119,0.029569,0.018148


In [93]:
isotinic_cols = [l+'_i' for l in labels]

In [94]:
calibrated = df[common_cols + isotinic_cols]

In [95]:
calibrated.head()

,site,actual,predicted,predicted_prob,adv_i,aggressive_i,alcohol_i,anonvpn_i,automobile_i,costtraps_i,...,tracker_i,updatesites_i,urlshortener_i,violence_i,warez_i,weapons_i,webmail_i,webphone_i,webradio_i,webtv_i
0,neptunesoccer,recreation,recreation,0.040239,0.000485,0.000097,0.000208,0.000000,0.000711,0.000000,...,0.000015,0.000000,0.000145,0.000094,0.000229,0.000376,0.000209,0.000024,0.000219,0.000098
1,carena-girls,porn,recreation,0.027002,0.001250,0.000180,0.001623,0.000000,0.001047,0.000297,...,0.000658,0.000132,0.001177,0.000314,0.000406,0.000429,0.001650,0.000024,0.001707,0.000490
2,natlhistoricseaport,recreation,gamble,0.022758,0.004717,0.001247,0.019231,0.002845,0.013193,0.002597,...,0.005998,0.005170,0.001892,0.000752,0.004912,0.002587,0.001879,0.009009,0.001707,0.001827
3,gta-sanandreas,hobby,hobby,0.048474,0.000109,0.000035,0.000020,0.000000,0.000047,0.000000,...,0.000010,0.000000,0.000000,0.000000,0.000000,0.000085,0.000069,0.000000,0.000050,0.000011
4,radionaba,webradio,webradio,0.029569,0.001250,0.000180,0.000517,0.000000,0.000944,0.000169,...,0.000015,0.000063,0.001177,0.000170,0.000365,0.000376,0.001009,0.000024,0.578431,0.000490


In [96]:
calibrated['predicted_prob'] = np.amax(calibrated[isotinic_cols].to_numpy(), axis=1)

<ipython-input-96-6acfe30f3a0a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calibrated['predicted_prob'] = np.amax(calibrated[isotinic_cols].to_numpy(), axis=1)


In [97]:
calibrated

,site,actual,predicted,predicted_prob,adv_i,aggressive_i,alcohol_i,anonvpn_i,automobile_i,costtraps_i,...,tracker_i,updatesites_i,urlshortener_i,violence_i,warez_i,weapons_i,webmail_i,webphone_i,webradio_i,webtv_i
0,neptunesoccer,recreation,recreation,0.830754,0.000485,0.000097,0.000208,0.000000,0.000711,0.000000,...,0.000015,0.000000,0.000145,0.000094,0.000229,0.000376,0.000209,0.000024,0.000219,0.000098
1,carena-girls,porn,recreation,0.404106,0.001250,0.000180,0.001623,0.000000,0.001047,0.000297,...,0.000658,0.000132,0.001177,0.000314,0.000406,0.000429,0.001650,0.000024,0.001707,0.000490
2,natlhistoricseaport,recreation,gamble,0.208955,0.004717,0.001247,0.019231,0.002845,0.013193,0.002597,...,0.005998,0.005170,0.001892,0.000752,0.004912,0.002587,0.001879,0.009009,0.001707,0.001827
3,gta-sanandreas,hobby,hobby,0.973730,0.000109,0.000035,0.000020,0.000000,0.000047,0.000000,...,0.000010,0.000000,0.000000,0.000000,0.000000,0.000085,0.000069,0.000000,0.000050,0.000011
4,radionaba,webradio,webradio,0.578431,0.001250,0.000180,0.000517,0.000000,0.000944,0.000169,...,0.000015,0.000063,0.001177,0.000170,0.000365,0.000376,0.001009,0.000024,0.578431,0.000490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263597,jades,porn,news,0.545809,0.001250,0.000180,0.000517,0.000000,0.000944,0.000047,...,0.000015,0.000000,0.000189,0.000170,0.000276,0.000376,0.000957,0.000024,0.000622,0.000255
263598,pornstrapon,porn,porn,0.962932,0.000215,0.000048,0.000094,0.000000,0.000173,0.000000,...,0.000010,0.000000,0.000145,0.000087,0.000019,0.000146,0.000092,0.000000,0.000050,0.000037
263599,awatunahomestead.co,recreation,recreation,0.998717,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
263600,sportsequip.co,shopping,shopping,0.808140,0.000485,0.000097,0.000208,0.000000,0.000711,0.000047,...,0.000015,0.000000,0.000145,0.000094,0.000229,0.000376,0.000209,0.000024,0.000219,0.000098


In [98]:
calibrated.to_csv('calibrated.csv')

In [99]:
uncalibrated.to_csv('uncalibrated.csv')

In [100]:
calibrated[calibrated['actual'] == calibrated['predicted']]

,site,actual,predicted,predicted_prob,adv_i,aggressive_i,alcohol_i,anonvpn_i,automobile_i,costtraps_i,...,tracker_i,updatesites_i,urlshortener_i,violence_i,warez_i,weapons_i,webmail_i,webphone_i,webradio_i,webtv_i
0,neptunesoccer,recreation,recreation,0.830754,0.000485,0.000097,0.000208,0.0,0.000711,0.000000,...,0.000015,0.000000,0.000145,0.000094,0.000229,0.000376,0.000209,0.000024,0.000219,0.000098
3,gta-sanandreas,hobby,hobby,0.973730,0.000109,0.000035,0.000020,0.0,0.000047,0.000000,...,0.000010,0.000000,0.000000,0.000000,0.000000,0.000085,0.000069,0.000000,0.000050,0.000011
4,radionaba,webradio,webradio,0.578431,0.001250,0.000180,0.000517,0.0,0.000944,0.000169,...,0.000015,0.000063,0.001177,0.000170,0.000365,0.000376,0.001009,0.000024,0.578431,0.000490
5,store.babycenter,shopping,shopping,0.974052,0.000121,0.000035,0.000020,0.0,0.000071,0.000000,...,0.000010,0.000000,0.000000,0.000000,0.000000,0.000085,0.000070,0.000000,0.000050,0.000011
6,ammres.co,shopping,shopping,0.704082,0.002227,0.000180,0.000517,0.0,0.000944,0.000047,...,0.000015,0.000000,0.000189,0.000170,0.000406,0.000376,0.001650,0.000024,0.001707,0.000255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263596,docbao.com,news,news,0.959677,0.000121,0.000040,0.000020,0.0,0.000157,0.000000,...,0.000010,0.000000,0.000142,0.000000,0.000019,0.000085,0.000092,0.000000,0.000219,0.000037
263598,pornstrapon,porn,porn,0.962932,0.000215,0.000048,0.000094,0.0,0.000173,0.000000,...,0.000010,0.000000,0.000145,0.000087,0.000019,0.000146,0.000092,0.000000,0.000050,0.000037
263599,awatunahomestead.co,recreation,recreation,0.998717,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
263600,sportsequip.co,shopping,shopping,0.808140,0.000485,0.000097,0.000208,0.0,0.000711,0.000047,...,0.000015,0.000000,0.000145,0.000094,0.000229,0.000376,0.000209,0.000024,0.000219,0.000098


In [101]:
calibrated[calibrated['actual'] != calibrated['predicted']]

,site,actual,predicted,predicted_prob,adv_i,aggressive_i,alcohol_i,anonvpn_i,automobile_i,costtraps_i,...,tracker_i,updatesites_i,urlshortener_i,violence_i,warez_i,weapons_i,webmail_i,webphone_i,webradio_i,webtv_i
1,carena-girls,porn,recreation,0.404106,0.001250,0.000180,0.001623,0.000000,0.001047,0.000297,...,0.000658,0.000132,0.001177,0.000314,0.000406,0.000429,0.001650,0.000024,0.001707,0.000490
2,natlhistoricseaport,recreation,gamble,0.208955,0.004717,0.001247,0.019231,0.002845,0.013193,0.002597,...,0.005998,0.005170,0.001892,0.000752,0.004912,0.002587,0.001879,0.009009,0.001707,0.001827
9,foxdenlane,shopping,recreation,0.490132,0.002232,0.000180,0.000517,0.000000,0.000944,0.000169,...,0.000658,0.000000,0.000703,0.000170,0.000276,0.000429,0.002952,0.000024,0.000622,0.000124
16,wusc.sc,news,radiotv,0.463235,0.000727,0.000180,0.000517,0.000000,0.000944,0.000169,...,0.000015,0.000063,0.000703,0.000170,0.000365,0.000376,0.000957,0.000024,0.463235,0.000637
20,brainvibe,shopping,recreation,0.466926,0.000727,0.000180,0.000517,0.000000,0.000944,0.000047,...,0.000015,0.000000,0.000189,0.000170,0.000276,0.000376,0.000957,0.000024,0.000640,0.000255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263572,krankenhaus-havelhoehe,hospitals,porn,0.835218,0.002232,0.000097,0.000517,0.000000,0.000937,0.000047,...,0.000015,0.000000,0.001129,0.000170,0.000276,0.000376,0.001650,0.000024,0.000622,0.000124
263580,sexy23,porn,recreation,0.268754,0.004717,0.000180,0.004224,0.000039,0.019295,0.000441,...,0.001166,0.000233,0.001892,0.000314,0.000406,0.000926,0.002952,0.000170,0.001247,0.000490
263581,darkhorse.net,shopping,porn,0.404106,0.120287,0.000463,0.000744,0.000797,0.001047,0.001391,...,0.023739,0.000362,0.003799,0.000661,0.004124,0.000698,0.056399,0.000450,0.001707,0.001827
263587,dtibasketball,shopping,recreation,0.756074,0.000507,0.000097,0.000215,0.000000,0.000937,0.000047,...,0.000015,0.000000,0.000145,0.000094,0.000229,0.000376,0.000575,0.000024,0.000219,0.000098


In [103]:
%ls -ltr *.sav | wc -l

54
